In [ ]:
using JLD2
using Flux
using MLJ
using MLJLIBSVMInterface

include("../src/filters.jl")
include("../src/reduction.jl")

@JLD2.load "variables/train_dataset.jld2" train_inputs train_targets
@JLD2.load "variables/test_dataset.jld2" test_inputs test_targets

In [ ]:
function buildANN(
    n_inputs::Int,
    hidden_layers::Vector{Int},
    n_outputs::Int;
    activation = relu
)
    layers = Any[]

    # capas ocultas
    in_dim = n_inputs
    for h in hidden_layers
        push!(layers, Dense(in_dim, h, activation))
        in_dim = h
    end

    # capa de salida
    push!(layers, Dense(in_dim, n_outputs))

    return Chain(layers...)
end

In [ ]:
function trainANN(ann::Chain, dataset::Tuple{DataFrame, BitArray};
    epochs::Int=1000, learningRate::Real=0.001, minLossChange::Real=1e-7, lossChangeWindowSize::Int=5)

    inputs = dataset[1];
    targets = dataset[2];

    loss(model, x, y) = Flux.Losses.crossentropy(model(x), y);
    trainingLosses = Float32[loss(ann, inputs, targets)];
    opt_state = Flux.setup(Adam(learningRate), ann);

    for epoch in 1:epochs
        Flux.train!(loss, ann, [(inputs, targets)], opt_state);
        current_loss = Float32(loss(ann, inputs, targets));
        push!(trainingLosses, current_loss);
    end;

    return trainingLosses
end;

In [ ]:
function createSVM(kernel::String, C::Real; gamma::Real=1.0, degree::Int=3, coef0::Real=0.0)

    # deshacemos One-Hot-Encoding 
    inputs, labels = dataset; 
    targets = [sum(findall(row)) for row in eachrow(labels)]

    k = kernel == "linear" ? LIBSVM.Kernel.Linear :
    kernel == "rbf"   ? LIBSVM.Kernel.RadialBasis :
    kernel == "poly"  ? LIBSVM.Kernel.Polynomial :
    kernel == "sigmoid" ? LIBSVM.Kernel.Sigmoid :

    model = SVMClassifier(
        kernel = kernel,
        cost = Float64(C),
        gamma = Float64(gamma),
        degree = Int32(degree),
        coef0 = Float64(coef0)
    )

    mach = machine(
        model,
        MLJ.table(inputs),
        categorical(targets)
    )

    MLJ.fit!(mach)
    return mach
end

In [ ]:
function createKNN(dataset::Tuple{DataFrame, BitArray}, k::Int = 5)

    # Deshacemos One-Hot-Encoding   
    inputs, labels = dataset; 
    targets = [sum(findall(row)) for row in eachrow(labels)]

    model = KNNClassifier(K = k)

    mach = machine(
        model,
        MLJ.table(inputs),
        categorical(targets)
    )

    fit!(mach)
    return mach
end

In [ ]:
# ypred = predict(svm, MLJ.table(Xtest))
# ypred = predict(kic nn, MLJ.table(Xtest))